In [17]:
import os
import re
import pandas as pd
from pdfminer.high_level import extract_text
from ipywidgets import interact, widgets, IntProgress, Label
from IPython.display import display, clear_output

In [26]:
# Hàm để đếm số lần xuất hiện của các từ khóa
def count_word_occurrences(file_path, target_words, progress_bar, status_label):
    # Đọc nội dung từ file PDF
    with open(file_path, 'rb') as file:
        text = extract_text(file)

    # Chuyển nội dung thành chữ thường
    content_lower = text.lower()

    # Tạo danh sách kết quả
    occurrences = {}
    for i, target_word in enumerate(target_words):
        # Cập nhật thanh tiến trình và trạng thái
        progress_bar.value = (i + 1) / len(target_words)
        status_label.value = f"Đang kiểm tra từ khóa '{target_word}'..."
        
        # Chuyển từ khóa thành chữ thường và tìm kiếm
        target_word_lower = target_word.lower()
        pattern = re.compile(r'\b{}\b'.format(re.escape(target_word_lower)))
        count = len(re.findall(pattern, content_lower))
        occurrences[target_word] = count
        
    return occurrences

# Tự động lấy danh sách file PDF trong thư mục
directory_path = "DemoPDF/"  # Thư mục chứa các file PDF
pdf_files = [f for f in os.listdir(directory_path) if f.endswith('.pdf')]

# Đọc danh sách từ khóa từ file txt
txt_file = "keywords.txt"
with open(txt_file, "r", encoding="utf-8") as file:
    keywords = [line.strip() for line in file.readlines()]

# Thiết lập giao diện
pdf_file_dropdown = widgets.Dropdown(options=pdf_files, description='Select PDF File')
submit_button = widgets.Button(description='Submit')
progress_bar = widgets.FloatProgress(value=0.0, min=0.0, max=1.0, description='Progress:', bar_style='info')
status_label = widgets.Label(value="")

result_output = widgets.Output()

# Hàm xử lý khi bấm nút Submit
def on_submit_clicked(b):
    pdf_file = pdf_file_dropdown.value
    progress_bar.value = 0
    status_label.value = "Bắt đầu đếm từ khóa..."
    count_results = count_word_occurrences(os.path.join(directory_path, pdf_file), keywords, progress_bar, status_label)
    
    # Hiển thị kết quả và chuẩn bị dữ liệu cho file Excel
    new_results = [{"PDF File": pdf_file, "Keyword": keyword, "Count": count} for keyword, count in count_results.items()]
    new_df = pd.DataFrame(new_results)
    
    output_file = "all_pdf_keyword_counts.xlsx"
    
    # Nếu file Excel đã tồn tại, thêm dữ liệu mới vào sau các dữ liệu cũ
    if os.path.exists(output_file):
        existing_df = pd.read_excel(output_file)
        combined_df = pd.concat([existing_df, new_df], ignore_index=True)
        combined_df.to_excel(output_file, index=False)
    else:
        # Nếu file chưa tồn tại, tạo file mới
        new_df.to_excel(output_file, index=False)
    
    with result_output:
        result_output.clear_output()
        print(f"Kết quả đã được lưu vào file Excel: {output_file}")
    
    status_label.value = "Hoàn thành!"

# Gán hàm xử lý vào nút
submit_button.on_click(on_submit_clicked)

# Hiển thị giao diện
display(pdf_file_dropdown, submit_button, progress_bar, status_label, result_output)

Dropdown(description='Select PDF File', options=('CAG_Baocaothuongnien_2018.pdf', 'CMP_Baocaothuongnien_2021.p…

Button(description='Submit', style=ButtonStyle())

FloatProgress(value=0.0, bar_style='info', description='Progress:', max=1.0)

Label(value='')

Output()